# Protein Design
## Technical Test

**Author:**  
**Dr Afroditi-Maria Zaki**

**Date:**  
**13.02.2024**

## 1. Retrieve protein structures

In [1]:
import urllib.request

def save_pdb(PDB):
    """
    Function that will search the RCSB PDB database and
    download and save the requested .pdb files
    """
    
    try:
        url = "https://files.rcsb.org/download/"
        filename = f"{PDB}.pdb"
        urllib.request.urlretrieve(url + filename, filename)
    except:
        print("Error: url does not exist!")


In [2]:
# Save PDB files required for this exercise
save_pdb("6M0J")
save_pdb("7Z0X")

## 2. Visualize protein complexes

In [3]:
import nglview as nv
import MDAnalysis as mda

def visualize_protein(PDB, sel1="protein", sel2=None, residues1=None, residues2=None, color1="orange", color2="magenta"):
    """
    Function that uses nglview to visualize the protein complexes
    """

    u = mda.Universe(PDB)
    view = nv.show_mdanalysis(u)
    view.gui_style = 'ngl'
    view.clear_representations()
    view.add_cartoon(selection=sel1, color=color1)
    view.add_cartoon(selection=sel2, color=color2)
    view.add_representation("ball+stick", selection="water", color="blue")
    if residues1 is not None:
        view.add_licorice(selection=residues1, color=color1)
    if residues2 is not None:
        view.add_licorice(selection=residues2, color=color2)

    return view

In [4]:
# View first protein complex
visualize_protein("6M0J.pdb", sel1=":E", sel2=":A")

ThemeManager()

NGLWidget(gui_style='ngl')

![6m0j](6m0j.png) 

In [5]:
# View second protein complex
visualize_protein("7Z0X.pdb", sel1=":R", sel2=":H or :L")

NGLWidget(gui_style='ngl')

![7Z0X](7z0x.png) 

## 3. Identify interface interactions

### Tailor-made script

In [8]:
import MDAnalysis as mda
from MDAnalysis.analysis import contacts
import numpy as np 
from numpy.linalg import norm

def distance(i, j):
    """
    Function that calculates the distance between a pair of atoms
    """
    d=norm(i.position-j.position)
    return round(d, 2), i.resid, i.resname, j.resid, j.resname         #returns the distance and the resid the atoms belong to.


def interface_contacts(structure, sel1, sel2, cutoff=4.0):
    """
    Function that analyses the contacts between two groups of atoms (e.g. two chains)
    and creates a set of the interface residues of each protein that form intermolecular interactions
    """

    # Define object of protein complex
    u = mda.Universe(structure)

    # Define atom selections of interest
    protein_sel1 = u.select_atoms(sel1)
    protein_sel2 = u.select_atoms(sel2)

    interface1, interface2 = [], []
    for atom1 in protein_sel1:
        for atom2 in protein_sel2:
            if distance(atom1, atom2)[0] <= cutoff:
                print("SARS CoV2 Spike protein S1 residue: ",distance(atom1, atom2)[1], distance(atom1, atom2)[2], "Binding protein residue: ", distance(atom1, atom2)[3], distance(atom1, atom2)[4], "Distance =",distance(atom1, atom2)[0], "A")
                interface1.append((distance(atom1, atom2)[1]))
                interface2.append((distance(atom1, atom2)[3]))
                
    return set(interface1), set(interface2)

In [9]:
# Interface amino acids of SARS CoV2 Spike protein S1 of 6M0J.pdb:
prot_A1 = interface_contacts("6M0J.pdb", "chainID E and not resname HOH", "chainID A and not resname HOH")[0]
# Interface amino acids of ACE2:
prot_B = interface_contacts("6M0J.pdb", "chainID E and not resname HOH", "chainID A and not resname HOH")[1]
# Interface amino acids of SARS CoV2 Spike protein S1 of 7Z0X.pdb:
prot_A2 = interface_contacts("7Z0X.pdb", "chainID R and not resname HOH", "protein and chainID H L and not resname HOH")[0]
# Interface amino acids of Antibody P5A-3C8:
prot_C = interface_contacts("7Z0X.pdb", "chainID R and not resname HOH", "protein and chainID H L and not resname HOH")[1]

print("Interface amino acids of SARS CoV2 Spike protein of 6M0J.pdb: \n")
print(prot_A1, "\n")

print("Interface amino acids of ACE2 of 6M0J.pdb: \n")
print(prot_B, "\n")

print("Interface amino acids of SARS CoV2 Spike protein of 7Z0X.pdb: \n")
print(prot_A2, "\n")

print("Interface amino acids of Antibody P5A-3C8 of 7Z0X.pdb: \n")
print(prot_C, "\n")

SARS CoV2 Spike protein S1 residue:  417 LYS Binding protein residue:  30 ASP Distance = 3.98 A
SARS CoV2 Spike protein S1 residue:  417 LYS Binding protein residue:  30 ASP Distance = 3.56 A
SARS CoV2 Spike protein S1 residue:  417 LYS Binding protein residue:  30 ASP Distance = 3.73 A
SARS CoV2 Spike protein S1 residue:  417 LYS Binding protein residue:  30 ASP Distance = 2.9 A
SARS CoV2 Spike protein S1 residue:  446 GLY Binding protein residue:  42 GLN Distance = 3.24 A
SARS CoV2 Spike protein S1 residue:  449 TYR Binding protein residue:  38 ASP Distance = 3.99 A
SARS CoV2 Spike protein S1 residue:  449 TYR Binding protein residue:  38 ASP Distance = 4.0 A
SARS CoV2 Spike protein S1 residue:  449 TYR Binding protein residue:  38 ASP Distance = 3.53 A
SARS CoV2 Spike protein S1 residue:  449 TYR Binding protein residue:  38 ASP Distance = 3.54 A
SARS CoV2 Spike protein S1 residue:  449 TYR Binding protein residue:  42 GLN Distance = 4.0 A
SARS CoV2 Spike protein S1 residue:  449 TY

### PLIP (Protein-Ligand Interaction Profiler)

For this step, I could also use PLIP  (https://github.com/pharmai/plip) to analyse the intermolecular interactions and the type of each interaction.

In [ ]:
from plip.structure.preparation import PDBComplex

HetID = 
Chain = 
Position = 

my_mol = PDBComplex()
my_mol.load_pdb("6M0J.pdb") # Load the PDB file into PLIP class
print(my_mol) # Shows name of structure and ligand binding sites
my_bsid = f"{HetID}:{Chain}:{Position}" # Unique binding site identifier (HetID:Chain:Position)
my_mol.analyze()
my_interactions = my_mol.interaction_sets[my_bsid] # Contains all interaction data


In [10]:
# Visualize protein complex with interface residues in licorice
visualize_protein("6M0J.pdb", sel1=":E", sel2=":A", residues1=[prot_A1], residues2=[prot_B])

NGLWidget(gui_style='ngl')

![6M0J_interface](6m0j_interface.png) 

In [11]:
# Visualize protein complex with interface residues 
visualize_protein("7Z0X.pdb", sel1=":R", sel2=":H or :L", residues1=[prot_A2], residues2=[prot_C])

NGLWidget(gui_style='ngl')

![7Z0X_interface](7z0x_interface.png) 

## 4. Conservation Analysis

Conservation Analysis could be used to identify functionally important residues, that are evolutionarily conserved across homolog proteins.    

By avoiding conserved residues that are essential for other functions or interactions, you can design mutations that selectively disrupt the undesired protein-protein interaction without affecting the overall function of the protein.  

To do that, I could use BioPython (https://biopython.org/).

In [ ]:
# ToDo

## 5. Mutate interface residues

In SARS CoV2 Spike protein of 6M0J.pdb, Lys417 forms a salt bridge with Asp30 of ACE2.
Lys417 is not involved in an intermolecular interaction with Antibody P5A-3C8 of 7Z0X.pdb, therefore, mutating this to Ala will decrease the binding affinity of prot_A with prot_B, but will not affect he binding affinity of prot_A with prot_C.  

This step should be automated. ML could be used to identify which amino acids should be mutated to decrease binding affinity with ACE2m while increasing binding affinity with the antibody.

In [12]:
import pymol

# Load the protein structure
# SARS CoV2 Spike protein with ACE2
pymol.cmd.load('6M0J.pdb', '6m0j')

# Mutate residues
def mutate_residue(residue, mutant):
    """
    Function that mutates selected residues
    """
    pymol.cmd.wizard("mutagenesis")
    pymol.cmd.do("refresh_wizard")
    pymol.cmd.get_wizard().set_mode(mutant)
    pymol.cmd.get_wizard().do_select(residue)
    pymol.cmd.get_wizard().apply()
    pymol.cmd.get_wizard()

    
# Mutate selected residues to alanine
mutations = [417]
for resid in mutations:
    mutate_residue(f"/6m0j//E/{resid}", 'ALA')

# Save mutant .pdb file
pymol.cmd.save('mutated_6M0J.pdb', '6m0j')

Selected!PyMOL>refresh_wizard

 ExecutiveRMSPairs: RMSD =    0.025 (4 to 4 atoms)
 Mutagenesis: no rotamers found in library.


In [13]:
# Load the protein structure 
# SARS CoV2 Spike protein with Antibody P5A-3C8
pymol.cmd.load('7Z0X.pdb', '7z0x')

# Mutate residues
def mutate_residue(residue, mutant):
    """
    Function that mutates selected residues
    """
    pymol.cmd.wizard("mutagenesis")
    pymol.cmd.do("refresh_wizard")
    pymol.cmd.get_wizard().set_mode(mutant)
    pymol.cmd.get_wizard().do_select(residue)
    pymol.cmd.get_wizard().apply()
    pymol.cmd.get_wizard()

    
# Mutate selected residues to alanine
mutations = [417]
for resid in mutations:
    mutate_residue(f"/7z0x//R/{resid}", 'ALA')

# Save mutant .pdb file
pymol.cmd.save('mutated_7Z0X.pdb', '7z0x')

Selected!PyMOL>refresh_wizard

 ExecutiveRMSPairs: RMSD =    0.026 (4 to 4 atoms)
 Mutagenesis: no rotamers found in library.


## 6. Compute binding affinities

#### Predict binding affinity with prodigy:  

https://github.com/haddocking/prodigy

In [14]:
import os

def predict_binding_affinity(PDB):
    """
    Function that uses the prodigy tool to
    predict protein-protein binding affinities
    """
    output_file = os.system(f"prodigy {PDB}.pdb >> prodigy_{PDB}.txt")
    binding_affinity = os.system(f"tail -2 prodigy_{PDB}.txt | head -1")

    return binding_affinity
              
# Binding affinity of SARS CoV2 Spike protein with ACE2 - Wild-Type
print("Binding affinity of SARS CoV2 Spike protein with ACE2 - Wild-Type")
predict_binding_affinity("6M0J")
# Binding affinity of SARS CoV2 Spike protein with ACE2 - Mutant
print("Binding affinity of SARS CoV2 Spike protein with ACE2 - Mutant")
predict_binding_affinity("mutated_6M0J")

# Binding affinity of SARS CoV2 Spike protein with Antibody P5A-3C8 - Wild-Type
print("Binding affinity of SARS CoV2 Spike protein with Antibody P5A-3C8 - Wild-Type")
predict_binding_affinity("7Z0X")
# Binding affinity of SARS CoV2 Spike protein with Antibody P5A-3C8 - Mutant
print("Binding affinity of SARS CoV2 Spike protein with Antibody P5A-3C8 - Mutant")
predict_binding_affinity("mutated_7Z0X")

Binding affinity of SARS CoV2 Spike protein with ACE2 - Wild-Type
[++] Predicted binding affinity (kcal.mol-1):    -11.9
Binding affinity of SARS CoV2 Spike protein with ACE2 - Mutant
[++] Predicted binding affinity (kcal.mol-1):    -11.7
Binding affinity of SARS CoV2 Spike protein with Antibody P5A-3C8 - Wild-Type
[++] Predicted binding affinity (kcal.mol-1):    -19.5
Binding affinity of SARS CoV2 Spike protein with Antibody P5A-3C8 - Mutant
[++] Predicted binding affinity (kcal.mol-1):    -19.5


0